In [423]:
import pandas as pd
from pymongo import MongoClient

# Load and clean CSV
data = pd.read_csv('/Users/hafiz/Downloads/listings.csv')

#####################     Listings Collection      ############################

df = data.copy()

# Clean and convert the price field
df["price"] = df["price"].str.replace("$", "", regex=False).str.replace(",", "", regex=False)
df["price"] = pd.to_numeric(df["price"], errors="coerce")  # Convert to float, NaN if failed

# Choose first 10 columns to keep
columns_to_use = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source',
                  'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'price', 'host_name', 'accommodates']
df = df[columns_to_use]

# Convert to list of dicts for MongoDB
records = df.to_dict(orient="records")

#######################     End Listings Collection     #############################


#######################     Hosts Collection       #################################


hosts = data[['host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost']].copy()

hosts = hosts.to_dict(orient='records')


#######################     End Hosts Collection    #################################




# Connect to MongoDB
client = MongoClient('mongodb:#Include Your Local Host Address Here#')
db = client['airbnb']

# Create a NEW collection (or overwrite if already exists)
collection = db['listings_cleaned']
hosts_collection = db['hosts']

# Optional: Clear existing data in that collection
collection.delete_many({})
hosts_collection.delete_many({})

# Insert cleaned data
collection.insert_many(records)
hosts_collection.insert_many(hosts)

print("Listings data inserted into 'collection' Collection.")
print("Hosts data inserted into 'hosts_collection' Collection.")

# print(df.columns)
print(collection.find_one({"last_scraped": "2025-03-07"}))
print("\n")
print(hosts_collection.find_one({"host_name":"Ruth"}))


for doc in hosts_collection.find().limit(5):
    print(doc)

print(db.list_collection_names())

Listings data inserted into 'collection' Collection.
Hosts data inserted into 'hosts_collection' Collection.
{'_id': ObjectId('6843733f38c4fbe31c5d19b5'), 'id': 1526583, 'listing_url': 'https://www.airbnb.com/rooms/1526583', 'scrape_id': 20250307145402, 'last_scraped': '2025-03-07', 'source': 'city scrape', 'name': 'Cozy and friendly home!', 'description': 'Very close to shopping malls, Main Street and close to bus stops.', 'neighborhood_overview': 'This side by side home is very close to many bus stops and groceries store, Walmart, Canadian Tire, Grant Park Mall and Pembina Hwy by Staford. 7.5kms away from UofM and UofW. I also host international students at my home.', 'picture_url': 'https://a0.muscache.com/pictures/1d892075-465e-42b3-a9d8-be22f783a274.jpg', 'host_id': 8145510, 'price': 48.0, 'host_name': 'Ruth', 'accommodates': 1}


{'_id': ObjectId('6843733f38c4fbe31c5d200f'), 'host_id': 8145510, 'host_url': 'https://www.airbnb.com/users/show/8145510', 'host_name': 'Ruth', 'host_si

In [289]:
# import pandas as pd
# from pymongo import MongoClient

# # Load CSV as DataFrame
# data = pd.read_csv('/Users/hafiz/Downloads/listings.csv')
# selected_columns = df.columns[:10].tolist()

# df = data.copy()
# df["price"] = df["price"].str.replace("$", "", regex=False).str.replace(",", "", regex=False)
# df["price"] = df["price"].astype(float)


# # Select only the first 10 columns
# df_subset = df[selected_columns]  # all rows, first 10 columns

# # Convert to list of dicts (JSON-like)
# records = df_subset.to_dict(orient='records')

# # Connect to MongoDB
# client = MongoClient('mongodb:#Include your local host address here#')
# db = client['airbnb']
# collection = db['listings']

# #print(records[0])

# # Insert subset of data
# collection.insert_many(records)

# # Display first 5 documents
# # for doc in collection.find().limit(5):
# #     print(doc)

# # collection.find_one() #this is how to get the first data point in a NoSQL database

# #find documents where the name field contains "apartment" (case-insensitive)
# #.find_one is a function that retrieves the first document that matches the given filter
# #collection.find_one({"name": {"$regex": "apartment", "$options": "i"}}) 
# #collection.find_one({"accommodates" : 6}) #This finds one listing that can accomodate 6 people

# #This is how to get the first few 5 or n datapoints in NoSQL
# # for doc in collection.find().limit(5): 
# #     print(doc)
# #     print("\n\n\n")


# #collection.find_one({"last_scraped": "2025-03-07"}) #find all listings scraped on a specific date

# # #Find the first 3 listings that belong to a host
# # host_id = 8145510 #This is how we identify the host

# # results = collection.find({"host_id": host_id})
# # for doc in results.limit(3):
# #     print(doc)
# #     print("\n\n\n")


# # #find all listings where description includes the word beach
# # for doc in collection.find({"description": {"$regex": "beach", "$options": "i"}}): 
# #     print(doc)

# # #project only specific fields, (e.g. show name and listing_url only)
# # for doc in collection.find({}, {"name": 1, "listing_url": 1, "_id": 0}).limit(5): 
# #     print(doc)

# # #count how many listings are in the Ebby - Wentworth neighbourhood
# # collection.count_documents({"host_neighbourhood" : "Ebby - Wentworth"})

# #Group listings by host id and count how many listings each has
# pipeline = [
#     {"$group": {"_id": "$host_id", "count": {"$sum": 1}}},
#     {"$sort": {"count": -1}},
#     {"$limit": 5}
# ]
# for doc in collection.aggregate(pipeline):
#     print(doc)

# # #Find all unique values in the source field
# # collection.distinct("source")



# print(df.columns)
# collection.find_one({"last_scraped": "2025-03-07"})

{'_id': 114498849, 'count': 4950}
{'_id': 74203278, 'count': 3870}
{'_id': 72603724, 'count': 2250}
{'_id': 95776055, 'count': 1800}
{'_id': 538224257, 'count': 1710}
Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_mini

{'_id': ObjectId('684121271cec7f782d618413'),
 'id': 1526583,
 'listing_url': 'https://www.airbnb.com/rooms/1526583',
 'scrape_id': 20250307145402,
 'last_scraped': '2025-03-07',
 'source': 'city scrape',
 'name': 'Cozy and friendly home!',
 'description': 'Very close to shopping malls, Main Street and close to bus stops.',
 'neighborhood_overview': 'This side by side home is very close to many bus stops and groceries store, Walmart, Canadian Tire, Grant Park Mall and Pembina Hwy by Staford. 7.5kms away from UofM and UofW. I also host international students at my home.',
 'picture_url': 'https://a0.muscache.com/pictures/1d892075-465e-42b3-a9d8-be22f783a274.jpg',
 'host_id': 8145510,
 'host_url': 'https://www.airbnb.com/users/show/8145510',
 'host_name': 'Ruth',
 'host_since': '2013-08-14',
 'host_location': 'Canada',
 'host_about': '',
 'host_response_time': 'within an hour',
 'host_response_rate': '100%',
 'host_acceptance_rate': '92%',
 'host_is_superhost': 'f',
 'host_thumbnail_url'

In [119]:
#This returns the name and host_id from listings that can accomodate 5 people
pipeline = [
    {"$match": {"accommodates": 5}},  # WHERE
    {"$project": {"name": 1, "host_id": 1, "_id": 0}}  # SELECT
]

#This returns the top 5 hosts with the highest amount of listings
pipeline = [
    {"$group": {"_id": "$host_id", "count": {"$sum": 1}}},  # GROUP BY + COUNT
    {"$sort": {"count": -1}},  # ORDER BY
    {"$limit": 5}  # LIMIT
]

#This joins the hosts collection with the listing collection and displays the name of each listing with their host name
pipeline = [
    {
        "$lookup": {
            "from": "hosts",           # other collection
            "localField": "host_id",   # field in current collection
            "foreignField": "host_id", # field in "hosts"
            "as": "host_info"
        }
    },
    {"$unwind": "$host_info"},  # Flatten host_info array
    {"$project": {"name": 1, "host_info.host_name": 1, "_id": 0}}  # SELECT
]

#This returns the top ten neighbourhoods with the most amount of listings that can accomodate more than three people
pipeline = [
    {"$match": {"accommodates": {"$gte": 3}}},  # WHERE
    {"$group": {"_id": "$host_neighbourhood", "count": {"$sum": 1}}},  # GROUP BY
    {"$match": {"count": {"$gt": 10}}}  # HAVING
]

#If we create an index like this, then we can perform better searches on data items
collection.create_index([("name", "text"), ("description", "text")])
collection.find({"$text": {"$search": "apartment"}})


In [433]:
#sample NoSQL questions

pipeline = [
    {"$group": {"_id": "$host_id", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
]
for doc in collection.aggregate(pipeline):
    print(doc)

#find 5 host listing for hosts named john 
print("\nfind 5 host listing for hosts named john")
list_query = collection.find({"host_name":{"$regex" : "john", "$options" : "i"}},
                             { "_id":0, "host_name":1, "name":1, "listing_url" : 1, "price" : 1}).limit(5)
for doc in list_query:
    print(doc)

#five listings where the price per night is $50
print("\nfive listings where the price per night is $50")
list_query = collection.find({ "price" : {"$gt":100} } ,
                             {"_id":0, "name":1, "price":1, "host_name":1}).limit(5)
for doc in list_query:
    print(doc)

#find listings that either have apartment in their name or can accomodate more than 4 guests
print("\nListings that have apartment in their name or can accomodate more than 4 guests")
list_query = collection.find(
    { "$or" : [ {"name" :  {"$regex":"apartment", "$options": "i"}}, {"accommodates" : { "$gt" : 4 }} ] }, 
    {"name":1, "host_name":1, "accommodates":1, "_id":0}
).limit(5)
for doc in list_query:
    print(doc)

#find listings that can only accommodate 2 people and return the name, price and accommodates
print("\nThese are listings that can accomodate 2 people")
list_query = collection.find(
    {"accommodates" : { "$eq" : 2 }},
    {"name":1,"price":1,"accommodates":1,"_id":0}
).limit(5)
for doc in list_query:
    print(doc)

#Find the hosts with the most number of listings, sort the list and then select the top three
print("\nThe top three hosts with the highest amount of listings")
pipeline = [
    {"$group": {"_id":"$host_id","count" : {"$sum":1}}},
    {"$sort" : {"count" : -1}},
    {"$limit" : 3}
]

for doc in collection.aggregate(pipeline):
    print(doc)

#Joining two collections of data and then retrieving some information from each
print("\nJoining two collections of data and then retrieving information")
pipeline = [
    {
        "$lookup" :{
            "from" : "listings_cleaned", #hosts
            "localField" : "host_id",
            "foreignField" : "host_id",
            "as" : "airbnb"
        }
    },
    #{"$unwind" : "$airbnb"},
    {"$project": { "airbnb.host_id":1, "host_name":1, "airbnb.name":1, "airbnb.price":1, "host_response_rate":1, "_id":0 }},
    {"$limit":5}
]
#Left join simulated using hosts_collection as the left collection and in the "from":"listings_cleaned" is used as the right join parameter
for doc in hosts_collection.aggregate(pipeline): 
    print(doc)
    print("\n")


{'_id': 114498849, 'count': 55}
{'_id': 74203278, 'count': 43}
{'_id': 72603724, 'count': 25}
{'_id': 95776055, 'count': 20}
{'_id': 538224257, 'count': 19}

find 5 host listing for hosts named john
{'listing_url': 'https://www.airbnb.com/rooms/24717604', 'name': 'Private room close to stuff - Great reviews', 'price': 29.0, 'host_name': 'John'}
{'listing_url': 'https://www.airbnb.com/rooms/38853577', 'name': '💎⭐️Brooklands beauty-Minutes from Airport✈️', 'price': 88.0, 'host_name': 'John'}
{'listing_url': 'https://www.airbnb.com/rooms/46756725', 'name': 'Near Polo park/Airport Suite - #1', 'price': 121.0, 'host_name': 'John'}
{'listing_url': 'https://www.airbnb.com/rooms/49907603', 'name': '🌟 💎 Bronx Beauty-minutes from downtown🔑🏙', 'price': 90.0, 'host_name': 'John'}
{'listing_url': 'https://www.airbnb.com/rooms/1013119232771606027', 'name': 'Suburban Paradise', 'price': 84.0, 'host_name': 'Johnny'}

five listings where the price per night is $50
{'name': 'Contemporary 5 bedroom in St

In [ ]:
#This is a Document based NoSQL Database
{
  "name": "Alice",
  "orders": [
    {"item": "Book", "price": 20},
    {"item": "Pen", "price": 5}
  ]
}


In [ ]:
#This is a key value based NoSQL Database
"user123" : { "name": "Alice", "age": 30 }
"session_token" : "xyz456"


In [ ]:
#This is a Columnar based NoSQL Database
ProductID | Category | Sales_US | Sales_EU
------------------------------------------
001       | Shoes    | 5000     | 4000
002       | Bags     | 7000     | 6000


In [ ]:
#This is a graph based NoSQL Database
(Alice) —[FRIEND]—> (Bob)
(Bob) —[LIKES]—> (Pizza)
